# Metodologías de cálculo de VaR y CVaR de un portafolio

Anteriormente vimos cómo calcular el VaR y CVaR de **un solo activo**. Ahora bien, ¿Cómo lo hacemos para un portafolio de **varios** activos?

Nos podemos apoyar en 3 métodos:

* Simulación histórica (No paramétrico)
* Método de varianzas y covarianzas o Delta-Normal (Paramétrico)
* Monte-Carlo (No paramétrico)

Estos son los 3 métodos más usados, sin embargo nos concentraremos en los dos primeros, ya que el último tiene un coste computacional muy alto.

## Preparativos

### ¡Carguemos las librerías!

In [1]:
#Para operaciones
import numpy as np
#Para dataframes
import pandas as pd
#Para obtener los datos
import yfinance as yf
#Para graficar
import matplotlib as plt
#Para gráficas más bonitas
import seaborn as sns
sns.set() #Para usar el estilo de seaborn
#Para usar funciones de la distribución normal.
from scipy.stats import norm

### ¡A crear el portafolio!

Vamos a crear un portafolio. Primero, elijamos los activos:

En este caso, vamos a usar compañías pertenecientes al Down Jones Industrial Avergae[INDU](https://www.spglobal.com/spdji/es/indices/equity/dow-jones-industrial-average/#overview), el cual es el índice bursátil de referencia de la bolsa de valores de Nueva York. Este índice refleja la evolución de las 30 empresas industriales con mayor capitalización bursátil que cotizan en este mercado.  Está conformado por 30 compañías bien establecidas (empresas con alto nivel de liquidez, en marcado a esta condición se le conoce como [Blue chip](https://www.investopedia.com/terms/b/bluechipstock.asp#:~:text=our%20editorial%20policies-,What%20Is%20a%20Blue%20Chip%20Stock%3F,often%20paying%20dividends%20to%20investors.))

Este índice tiene cierta ponderación de cada uno de sus activos.



In [2]:
data=pd.read_excel("Data/INDU.xlsx")
data.head()

,Ticker,Name,Weight
0,AAPL UW Equity,Apple Inc,3.153727
1,AMGN UW Equity,Amgen Inc,4.880003
2,AXP UN Equity,American Express Co,3.519216
3,BA UN Equity,Boeing Co/The,3.448652
4,CAT UN Equity,Caterpillar Inc,4.216276


Arreglemos un poco la información, ya que solo nos interesa la primera parte del `Ticker`, el resto es info extra

In [3]:
data["Ticker"]=data["Ticker"].apply(lambda x: x.split()[0])
data.head()

,Ticker,Name,Weight
0,AAPL,Apple Inc,3.153727
1,AMGN,Amgen Inc,4.880003
2,AXP,American Express Co,3.519216
3,BA,Boeing Co/The,3.448652
4,CAT,Caterpillar Inc,4.216276


Inicializamos la clase de cada `Ticker`

In [4]:
info=[]
for i in range(len(data["Ticker"])):
    info.append(yf.Ticker(data.loc[i,"Ticker"]))

In [23]:
AAPL= yf.Ticker("AAPL")
AAPL= AAPL.history(start="2012-04-12",end="2015-12-10")
AAPL.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2012-04-12,19.113263,19.306843,18.975647,19.045067,614336800,0.0,0.0
2012-04-13,19.086050,19.104094,18.456077,18.508677,859644800,0.0,0.0
2012-04-16,18.656382,18.663110,17.683595,17.741087,1050786800,0.0,0.0
2012-04-17,17.704695,18.654546,17.489709,18.645372,1025528000,0.0,0.0
2012-04-18,18.768307,18.968004,18.431607,18.603781,954531200,0.0,0.0


Obtenemos la historia. Elegiremos una historia de 3 años

In [5]:
fecha_inicio="2019-04-23" #La fecha de término debe ser un día DESPUÉS, t+1 en formato aaaa-mm-dd
fecha_fin="2022-04-23"
for i in range(len(info)):
    info[i]=info[i].history(start=fecha_inicio, end=fecha_fin)
    info[i]=(info[i]).rename(columns={"Close": data.loc[i,"Ticker"]})
    info[i]=(info[i])[data.loc[i,"Ticker"]]
    info[i].head()

Juntamos todo en un solo portafolio

In [6]:
portfolio = pd.concat(info, axis=1, join='inner')
portfolio.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-23,50.505840,164.530655,109.329842,365.259827,132.336548,161.229996,51.850819,105.395042,131.749863,49.027340,...,64.673965,121.566399,85.082184,95.893654,128.470123,220.169296,158.690720,50.081024,47.406017,97.957260
2019-04-24,50.427940,163.194351,109.377808,366.666138,128.330048,159.559998,52.024605,102.164619,133.468857,47.615379,...,64.786659,121.149681,86.016403,96.386314,129.559250,219.949661,158.523834,50.019688,47.628704,98.394447
2019-04-25,49.970303,165.189682,111.162086,373.834198,126.839226,163.089996,51.521557,101.836395,135.583008,47.062504,...,66.182449,125.161819,85.208687,96.005188,129.568558,220.369797,158.062454,48.941822,46.925011,98.384933
2019-04-26,49.731739,166.095795,112.812057,371.871307,129.541306,165.960007,51.109962,101.145393,138.230667,46.994461,...,66.433861,125.879005,85.938545,98.403481,130.545975,226.299438,159.937393,49.581528,47.076439,96.493652
2019-04-29,49.807209,165.821213,112.006248,370.172028,129.532013,165.360001,51.338627,101.680916,137.618149,47.207108,...,66.563904,125.762680,85.889900,97.399544,131.681641,226.729111,161.134995,49.809364,47.708878,96.522171


### Obtenemos los pesos

In [7]:
pesos=data[["Weight","Ticker"]]
pesos=pesos.set_index('Ticker')
pesos=pesos.rename(columns={"Weight": "Peso"})
pesos.head()

,Peso
Ticker,
AAPL,3.153727
AMGN,4.880003
AXP,3.519216
BA,3.448652
CAT,4.216276


### Obtenemos los rendimientos diarios

In [8]:
rendimientos=[]
for i in portfolio.columns:
    precio_hoy=portfolio[i]
    precio_anterior=precio_hoy.shift(1)
    rendimiento_diario=np.log(precio_hoy/precio_anterior)
    rendimientos.append(rendimiento_diario)

In [9]:
rendimientos_diarios=pd.concat(rendimientos,axis=1)
rendimientos_diarios=rendimientos_diarios.dropna()
rendimientos_diarios.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-24,-0.001544,-0.008155,0.000439,0.003843,-0.030743,-0.010412,0.003346,-0.031130,0.012963,-0.029222,...,0.001741,-0.003434,0.010920,0.005124,0.008442,-0.000998,-0.001052,-0.001225,0.004686,0.004453
2019-04-25,-0.009117,0.012153,0.016181,0.019361,-0.011685,0.021882,-0.009716,-0.003218,0.015716,-0.011679,...,0.021316,0.032581,-0.009435,-0.003962,0.000072,0.001908,-0.002915,-0.021784,-0.014885,-0.000097
2019-04-26,-0.004786,0.005470,0.014734,-0.005265,0.021079,0.017445,-0.008021,-0.006809,0.019340,-0.001447,...,0.003792,0.005714,0.008529,0.024674,0.007515,0.026552,0.011792,0.012986,0.003222,-0.019410
2019-04-29,0.001516,-0.001655,-0.007169,-0.004580,-0.000072,-0.003622,0.004464,0.005281,-0.004441,0.004515,...,0.001956,-0.000925,-0.000566,-0.010255,0.008662,0.001897,0.007460,0.004585,0.013345,0.000296
2019-04-30,-0.019444,-0.010264,0.004017,-0.003594,0.002873,-0.000060,-0.003212,0.019683,-0.016868,0.021920,...,0.024826,0.006375,-0.004884,0.016094,0.016059,-0.018618,0.001704,0.006139,0.000187,0.012525


¡Listo! Ya creamos nuestro portafolio, ahora sí: ¡A obtener sus medidas de riesgo!

## Simulación histórica

Supongamos se tiene un portafolio $P$ con $m$ activos, y los precios de estos para $n$ días, sean:
* ${}_{k}S_{i}$ : precio del activo $k$ a tiempo $i \in \{1, . . . , n\}$, $k \in \{1, . . . , m\}$.
* ${}_{P}S_{i}$: valor del portafolio $P$ a tiempo $i$.
* ${}_{k}r_{i}$: rendimiento logarítmico del activo $k$ a tiempo $i$.
* ${}_{P}r_{i}$: redimiento del portafolio $P$ a tiempo $i$.
* $perc_{\alpha}(\beta)$ : percentil $\beta$ de la distribución empírica de observaciones de $\alpha$ .
* ${}_{k}w$ : peso del activo $k$ en el portafolio.

Así para $i \in \{2, . . . , n\}$, los rendimientos logarítmicos del activo $k$ son:

$${}_{k}r_{i} = ln(\frac{{}_{k}S_{i}}{{}_{k}S_{i-1}})$$ 

Luego:


$${}_{P}S_{i} = \sum_{k = 1}^{m} {}_{k}w \dot {}_{k}S_{i}$$ 
$${}_{P}r_{i} = \sum_{k = 1}^{m} {}_{k}w \dot {}_{k}r_{i}$$ 

Si $R$ es la variable aleatoria de los rendimientos logarítmicos de $P$, de la cuál tenemos las observaciones ${}_{P}r_{1}, . . . , {}_{P}r_{n}$ , entonces defimos el VaR de simulación histórica de un solo activo con distrtibución de ganancias $X$ como:

$$VaR_{\alpha}(X) = {}_{P}S_{n} - {}_{P}S_{n} *  e^{perc_{R}(1-\alpha)}$$

Para calcular el CVaR solo se debe considerar

$$z = promedio (  \{ {}_{P}r_{i} | {}_{P}r_{i} \leq perc_{R}(1-\alpha)\}) $$

Y asi: 

$$ CVaR_{\alpha}(X) = {}_{P}S_{n} - {}_{P}S_{n} * e^{z} $$

Obtenemos 

$${}_{P}S_{i} = \sum_{k = 1}^{m} {}_{k}w \dot {}_{k}S_{i}$$ 

In [10]:
precio_portafolio=portfolio.dot(pesos)
precio_portafolio=precio_portafolio.rename(columns={"Peso": "Precio"})
precio_portafolio.head()

,Precio
Date,
2019-04-23,14946.667341
2019-04-24,14904.986719
2019-04-25,14898.961812
2019-04-26,15008.749010
2019-04-29,15018.488093


Obtenemos

$${}_{P}r_{i} = \sum_{k = 1}^{m} {}_{k}w \dot {}_{k}r_{i}$$ 

In [11]:
rend_portafolio=rendimientos_diarios.dot(pesos)
rend_portafolio=rend_portafolio.rename(columns={"Peso": "Rendimiento"})
rend_portafolio.head()

,Rendimiento
Date,
2019-04-24,-0.327226
2019-04-25,-0.103855
2019-04-26,0.630576
2019-04-29,0.067668
2019-04-30,0.090712


Como estamos trabajando con la distribución de **rendimientos** y no la de pérdidas, debemos usar el cuantil $1-\alpha$ y no el cuantil $\alpha$, que es el que usaría **en caso de estar trabajando con la distribución de pérdidas** 

Si quisieramos trabjar con la distribución de pérdidas, esta la obtenemos como:

$L=(-1)*R$

Donde: 

* $L$ Es la distribución de pérdidas
* $R$ Es la distribución de rendimientos

Obtenemos
$$perc_{R}(1-\alpha)$$
Y también
$$_{P}S_{i}$$

In [12]:
alpha=0.95
perc=rend_portafolio.quantile(1-alpha)
last_value=precio_portafolio.iloc[-1,0]

Calculamos
$$VaR_{\alpha} = {}_{P}S_{n} - {}_{P}S_{n} *  e^{perc_{R}(1-\alpha)}$$

In [13]:
VaR=(last_value*np.exp(perc)-last_value)[0]
VaR

-19410.513999494295

Obtenemos:

$$z = promedio (  \{ {}_{P}r_{i} | {}_{P}r_{i} \leq perc_{R}(1-\alpha)\}) $$

In [14]:
z=rend_portafolio[rend_portafolio<=perc].mean()

Calculamos:

$$ CVaR_{\alpha} = {}_{P}S_{n} - {}_{P}S_{n} * e^{z} $$

In [15]:
CVaR=(last_value*np.exp(z)-last_value)[0]
CVaR

-22145.820329690458

## Método de varianzas y covarianzas (Delta-Normal)

Sea $X$ la variable aleatoria del precio de nuestro portafolio.

**Hipótesis**: Para este método, supondremos que $X$ tiene distribución normal, así:
$$VaR_{\alpha}(X) = {}_{P}S_{n} * Z_{(1−\alpha)} * \sigma $$

Donde $Z_{(1-\alpha)} = F_{N}^{-1}(1-\alpha)$ con $F_{N}$ la función de distribución de una normal estándar, Y también:

$$CVaR_{\alpha}(X) = {}_{P}S_{n} \frac{e^{-\frac{z_{1-\alpha}^{2}}{2}}}{(1-\alpha) \sqrt{2\pi}} * \sigma$$

Entonces el cálculo de esto valores ya solo dependen de la forma de calcular la desviación estándar $\sigma$ de $X$. En este caso se utilizará la matriz de varianzas y covarianzas ($V$) de los rendimientos logarítmicos de cada uno de los activos del portafolio:

$$\sigma = (w^{T} * V * w)^{(1/2)}$$

Con $w$ el vector de pesos de cada uno de los activos del portafolio (${}_{k}w$)

Procedemos a calcular la matriz de varianzas y covarianzas $V$

In [16]:
covarianzas=rendimientos_diarios.cov()
covarianzas

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
AAPL,0.000462,0.000187,0.000258,0.000332,0.000176,0.000305,0.000244,0.000205,0.000213,0.000249,...,0.000128,0.000332,0.000233,0.000146,0.000174,0.000226,0.000260,0.000088,0.000161,0.000130
AMGN,0.000187,0.000302,0.000168,0.000152,0.000146,0.000152,0.000171,0.000158,0.000124,0.000189,...,0.000151,0.000184,0.000116,0.000140,0.000144,0.000190,0.000159,0.000098,0.000153,0.000109
AXP,0.000258,0.000168,0.000718,0.000673,0.000361,0.000253,0.000272,0.000480,0.000396,0.000469,...,0.000157,0.000257,0.000301,0.000147,0.000375,0.000309,0.000389,0.000133,0.000255,0.000085
BA,0.000332,0.000152,0.000673,0.001322,0.000409,0.000331,0.000293,0.000568,0.000459,0.000555,...,0.000158,0.000302,0.000361,0.000148,0.000420,0.000322,0.000396,0.000136,0.000311,0.000090
CAT,0.000176,0.000146,0.000361,0.000409,0.000454,0.000151,0.000211,0.000337,0.000251,0.000434,...,0.000129,0.000160,0.000202,0.000112,0.000233,0.000200,0.000217,0.000117,0.000239,0.000087
CRM,0.000305,0.000152,0.000253,0.000331,0.000151,0.000610,0.000216,0.000193,0.000228,0.000243,...,0.000122,0.000345,0.000224,0.000113,0.000175,0.000194,0.000270,0.000075,0.000105,0.000110
CSCO,0.000244,0.000171,0.000272,0.000293,0.000211,0.000216,0.000373,0.000225,0.000224,0.000271,...,0.000134,0.000244,0.000197,0.000153,0.000181,0.000205,0.000225,0.000113,0.000213,0.000115
CVX,0.000205,0.000158,0.000480,0.000568,0.000337,0.000193,0.000225,0.000653,0.000294,0.000435,...,0.000174,0.000209,0.000226,0.000112,0.000337,0.000284,0.000288,0.000123,0.000213,0.000075
DIS,0.000213,0.000124,0.000396,0.000459,0.000251,0.000228,0.000224,0.000294,0.000478,0.000304,...,0.000104,0.000218,0.000236,0.000116,0.000210,0.000194,0.000279,0.000098,0.000172,0.000079
DOW,0.000249,0.000189,0.000469,0.000555,0.000434,0.000243,0.000271,0.000435,0.000304,0.000758,...,0.000157,0.000225,0.000273,0.000146,0.000346,0.000244,0.000301,0.000149,0.000296,0.000125


Pasamos los dataframes a objetos `numpy` para tratarlos como matrices y realizar el siguiente cálculo

In [17]:
cov_numpy=covarianzas.to_numpy()
pesos_numpy=pesos.to_numpy()

Aquí se calcula 

$$\sigma = (w^{T} * V * w)^{(1/2)}$$

In [18]:
prod_1=np.matmul(pesos_numpy.transpose(),cov_numpy)
prod_2=np.matmul(prod_1,pesos_numpy)
sigma=prod_2**0.5
sigma=float(sigma)

Obtenemos 

$$Z_{(1−\alpha)}$$

In [19]:
alpha=0.95
z_perc=norm.ppf(1-alpha)

Calculamos 

$$VaR_{\alpha}=_{P}S_{n} * Z_{(1−\alpha)} * \sigma$$

In [20]:
VaR_Cov=precio_portafolio.iloc[-1,0]*sigma*z_perc
VaR_Cov

-56228.97859379227

Calculamos

$$\frac{e^{-\frac{z_{1-\alpha}^{2}}{2}}}{(1-\alpha) \sqrt{2\pi}}$$

In [21]:
frac_CVaR=-np.exp(-z_perc**(2)/2)/((1-alpha)*(2*np.pi)**0.5)

Lo anterior lo multiplicamos por $_{P}S_{n}$ y por $\sigma$ para obtener:

$$CVaR_{\alpha}=_{P}S_{n} \frac{e^{-\frac{z_{1-\alpha}^{2}}{2}}}{(1-\alpha) \sqrt{2\pi}} * \sigma$$

In [22]:
CVaR_Cov=precio_portafolio.iloc[-1,0]*frac_CVaR*sigma
CVaR_Cov

-70513.40763581396

Bibliografía:
1. [Cerrato, M. (2012). The mathematics of derivatives securities with applications in MATLAB. John Wiley & Sons.](https://onlinelibrary.wiley.com/doi/pdf/10.1002/9781118467398.app3)